# Création des datasets des reviews anglaise et française

Basé sur le fichier kaggle suivant : https://www.kaggle.com/datasets/kieranpoc/steam-reviews/data

L'objectif est de constituer deux dataset d'environs 50 000 reviews équilibré 50/50 sur le sentiment.
Le fichier principale sur kaggle étant très volumineux on pourra se permettre de raffiner et de sous échantilloner.


Traitement du fichier contenant l'ensemble des reviews (environ 100 millions 40gb)
- Filtrage des reviews en anglais et français
- Filtrer avec weighted_score > 0.5
- Application des mêmes filtres que pour le fichier des 500 000 reviews
- Visualiser le nombre de reviews possible pour avoir un dataset anglais équilibré 50/50 posifif/négatif
- Visualiser le nombre de reviews possible pour avoir un dataset français / anglais équilibré (25% review positive anglais, 25% positive français etc.)

## Import des librairies

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re

import kagglehub

## Méthodes utiles

In [2]:
def column_summary(df: pd.DataFrame):
    summary = []
    for col in df.columns:
        col_type = df[col].dtype
        non_null = df[col].notna().sum()
        null_count = df[col].isna().sum()

        # Gérer le cas où la colonne contient des listes (unhashable)
        try:
            unique_count = df[col].nunique()
        except TypeError:
            # Si erreur (listes), convertir en string temporairement
            unique_count = df[col].astype(str).nunique()
            print(
                f"⚠️ Colonne '{col}' contient des types non-hashable (probablement des listes)"
            )

        summary.append(
            {
                "Column": col,
                "Type": str(col_type),
                "Non-Null Count": non_null,
                "Null Count": null_count,
                "Unique Values": unique_count,
            }
        )

    # Afficher le résumé des colonnes
    print("=" * 80)
    print("Résumé détaillé des colonnes:")
    print("=" * 80)
    column_summary_df = pd.DataFrame(summary)
    print(column_summary_df.to_string(index=False))
    print("\n")

In [3]:
def print_voted_up_count_proportion(df: pd.DataFrame):
    voted_up_counts = df["voted_up"].value_counts()
    total_reviews = len(df)

    print("voted_up (reviews positive:1 / negative:0)")
    for voted_up_value, count in voted_up_counts.items():
        proportion = (count / total_reviews) * 100
        print(
            f"voted_up = {voted_up_value}: Count = {count}, Proportion = {proportion:.2f}%"
        )

    print(
        f"\nPour obtenir une distribution équilibrée 50/50, en sous échantillonnant la classe majoritaire\n"
    )
    print(
        f"On aurait {min(voted_up_counts)} reviews par classe. Soit un total de {2 * min(voted_up_counts)} reviews."
    )

In [4]:
# clean html tags
def clean_tags(text: str):
    if not isinstance(text, str):
        return text
    text = re.sub(r"\[/?[a-zA-Z0-9]+[^\]]*\]", "", text)
    text = re.sub(r"</?[a-zA-Z0-9]+[^>]*>", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [5]:
# clean ascii art
def has_enough_letters(text: str, min_ratio=0.5):

    if not isinstance(text, str) or len(text) == 0:
        return False

    letters = sum(c.isalpha() for c in text)
    return (letters / len(text)) >= min_ratio

In [6]:
# équilibrage des classes par sous échantillonnage
def balance_classes(df: pd.DataFrame) -> pd.DataFrame:
    # Compter chaque classe
    count_0 = (df["voted_up"] == 0).sum()
    count_1 = (df["voted_up"] == 1).sum()

    # Sous-échantillonner la classe majoritaire pour égaler la minoritaire
    df_class_0 = df[df["voted_up"] == 0]
    df_class_1 = df[df["voted_up"] == 1].sample(
        n=count_0, random_state=42
    )

    # Combiner et mélanger
    df_balanced = (
        pd.concat([df_class_0, df_class_1])
        .sample(frac=1, random_state=42)
        .reset_index(drop=True)
    )
    return df_balanced

## Récupération du chemin local cache kaggle

In [7]:
# Download latest version
path = kagglehub.dataset_download("kieranpoc/steam-reviews")

print("Path to dataset files:", path)


def list_files_recursively(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            print(os.path.join(root, file))

Path to dataset files: /home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2


## Filtre des reviews

On a pas besoin d'autant de review pour le fine tuning, comme on a vue dans l'exploration du fichier des review anglaise on va pouvoir en filtrer pour garder les plus légitimes et qualitatives 
- minimiser celle généré par des bots, 
- taille minimum pour garder le sens 
- taille maximum pour éviter que la fin soit tronquée
- enlever les ascii art
- enlever les balises html

In [8]:
def clean_review_with_all_processing(df: pd.DataFrame,play_time_forever = 30,min_word_count=10, max_word_count=400):
    df_clean = df.copy()

    # filtre weighted_vote_score
    
    df_clean = df_clean[(df_clean["weighted_vote_score"] > 0.5)]

    # Filtre les reviews des joueurs légitimes
    df_clean = df_clean[
        (df_clean["steam_purchase"] == 1)  # Achat vérifié
        & (df_clean["received_for_free"] == 0)  # Pas reçu gratuitement
        & (df_clean["author_playtime_forever"] > play_time_forever)  # Au moins 30 minutes de jeu
    ]

    # Clean html tags
    df_clean["review"] = df_clean["review"].apply(clean_tags)

    # Filtre les reviews ASCII Art
    df_clean = df_clean[df_clean["review"].apply(has_enough_letters)]

    # filtre size min et max
    df_clean = df_clean[
        (df_clean["word_count"] >= min_word_count) & (df_clean["word_count"] <= max_word_count)
    ]
    
    # Supprimer reviews en double
    df_clean = df_clean.drop_duplicates(subset=["review"])
    df_clean = df_clean.reset_index(drop=True)

    return df_clean

## Création du fichier des reviews fr

### Script de récupération intermédiaire 

In [9]:
# Commande python src/create_fr_all_review_file.py 

### Chargement des données de review intermédiaire

In [10]:
df_reviews_fr = pd.read_csv(
    os.path.join(path, "fr_all_reviews.csv"), low_memory=True  # Optimize memory usage
)

df_reviews_fr.head(3)

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,word_count
0,148409645,10,Counter-Strike,76561199394155077,0,5,3263.0,2501.0,898,1.698276e+09,...,0,0,0.0,0,1,0,0,1,NaN,3.0
1,148041781,10,Counter-Strike,76561199494824658,26,5,406.0,195.0,208,1.698338e+09,...,0,0,0.0,0,0,0,0,1,NaN,1.0
2,148041449,10,Counter-Strike,76561199204466974,0,1,1064.0,819.0,192,1.698264e+09,...,0,0,0.0,0,1,0,0,1,NaN,1.0


### Quelques stats de vérification des données

In [11]:
df_reviews_fr["language"].value_counts()
print_voted_up_count_proportion(df_reviews_fr)

bins = [0, 10, 20, 50, 100, 300, 512, float("inf")]
labels = ["<10", "10-20", "20-50", "50-100", "100-300", "300-512", ">512"]
df_reviews_fr["tranche"] = pd.cut(df_reviews_fr["word_count"], bins=bins, labels=labels)

print("\nAffichage de toute les tranches de longueurs:")
print(df_reviews_fr["tranche"].value_counts().sort_index())

voted_up (reviews positive:1 / negative:0)
voted_up = 1: Count = 2253921, Proportion = 86.28%
voted_up = 0: Count = 358467, Proportion = 13.72%

Pour obtenir une distribution équilibrée 50/50, en sous échantillonnant la classe majoritaire

On aurait 358467 reviews par classe. Soit un total de 716934 reviews.

Affichage de toute les tranches de longueurs:
tranche
<10        1267104
10-20       378075
20-50       441165
50-100      248510
100-300     210392
300-512      41461
>512         25681
Name: count, dtype: int64


### Traitement du fichier fr

In [12]:
df_reviews_clean_fr = clean_review_with_all_processing(df_reviews_fr)

print(f"Avant : {len(df_reviews_fr)}")
print(f"Après : {len(df_reviews_clean_fr)}")

print_voted_up_count_proportion(df_reviews_clean_fr)

Avant : 2612388
Après : 187820
voted_up (reviews positive:1 / negative:0)
voted_up = 1: Count = 157610, Proportion = 83.92%
voted_up = 0: Count = 30210, Proportion = 16.08%

Pour obtenir une distribution équilibrée 50/50, en sous échantillonnant la classe majoritaire

On aurait 30210 reviews par classe. Soit un total de 60420 reviews.


Environ 60 000 review française c'est confortable pour passer au fine tuning des modèles BERT pour le Français

### Vérification de quelques review fr positive

In [13]:
for i, review in enumerate(
    df_reviews_clean_fr[
        (df_reviews_clean_fr["voted_up"] == 1)
        & (df_reviews_clean_fr["word_count"] <= 30)  # pas trop long pour relecture
    ]["review"].head(5),
    1,
):
    print(f"--- Review {i} ---")
    print(review)
    print()

--- Review 1 ---
Plus de 20 ans en arrière et déjà la meilleure franchise de FPS !

--- Review 2 ---
Je viens d'acheter le Season Pass 2020, c'est mauvais. En dehors de ça, le pilotage d'helicoptere est plutot correct si on aime manger. Je recommande.

--- Review 3 ---
Le classique de tous les FPS, la légende du multiplayer shooter. Les souvenirs que je garde de ce jeu sont innombrables! 10/10 !

--- Review 4 ---
C'est sympa mais une fois on m'a insulter de cheateur parce que je suis trop fort

--- Review 5 ---
Ce jeu est une légende. il n'a jamais perdu de reputation au cours des années.



### Vérification de quelques review fr négative

In [14]:
for i, review in enumerate(
    df_reviews_clean_fr[
        (df_reviews_clean_fr["voted_up"] == 0)
        & (df_reviews_clean_fr["word_count"] <= 30)  # pas trop long pour relecture
    ]["review"].head(5),
    1,
):
    print(f"--- Review {i} ---")
    print(review)
    print()

--- Review 1 ---
Bien que le concept de boucle temporelle et le design soient plaisants, le jeu est beaucoup trop court. De plus, les fins sont très répetitives et rendent le jeu monotone.

--- Review 2 ---
Not a twinstick gameplay :( Tee gameplay is absolutly stupid and not fun at all :(

--- Review 3 ---
Un jeu moyen, sans originalité, sans créativité, sans passion. Durée moins de 4 heures y compris le chapitre bonus. Histoire inexistante.

--- Review 4 ---
Encore un jeu ou l'on sert de beta testeur, en payant plein pot. Le jeu est actuellement fini à la pisse, bugué et extremement mal optimisé.

--- Review 5 ---
Très redondant. Les missions se ressemblent, et on refait souvent les mêmes ! c'est dommage parce que c'est lassant à la longue.



### Equilibrage des classes (50/50 positif/négatif)

In [15]:
df_balanced_fr = balance_classes(df_reviews_clean_fr)

print(f"\nDataset équilibré : {len(df_balanced_fr)} reviews (50/50)")
print(df_balanced_fr["voted_up"].value_counts())


Dataset équilibré : 60420 reviews (50/50)
voted_up
0    30210
1    30210
Name: count, dtype: int64


### Export du fichier des review fr pour fine tuning

In [16]:
# export du fichier des reviews fr
output_file_fr = os.path.join(path, "reviews_fr_processed.csv")
keep_columns = ["voted_up", "review", "weighted_vote_score"]

df_balanced_fr[keep_columns].to_csv(output_file_fr, index=False, encoding="utf-8")
print(f"Fichier des reviews fr nettoyées exporté vers : {output_file_fr}")

Fichier des reviews fr nettoyées exporté vers : /home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/reviews_fr_processed.csv


## Création du fichier des reviews en

### Script de récupération intermédiaire 

Un premier test de créer un fichier intermédiaire contenant exclusivement les reviews anglaises avec un weighted score vote > 0.5 a été réalisé mais celui-ci était très volumineux > 6GB. Un ajustement du weighted score à 0.6 nous donne un fichier plus raisonable et permettra de créer le dataset d'au moins 50 000 reviews

In [17]:
# Commande python src/create_en_weighted_score_above_06_file.py

### Chargement des données de review intermédiaire

In [18]:
df_reviews_en = pd.read_csv(
    os.path.join(path, "en_weighted_score_above_06.csv"), low_memory=True  # Optimize memory usage
)

df_reviews_en.head(3)

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,word_count
0,148630322,10,Counter-Strike,76561198801790858,30,4,39025.0,5.0,39025,1.697570e+09,...,7,1,0.608541,0,1,0,0,1,NaN,12.0
1,148235752,10,Counter-Strike,76561198092615109,0,9,767.0,76.0,706,1.698172e+09,...,10,1,0.632219,0,1,0,0,1,NaN,3.0
2,147776528,10,Counter-Strike,76561199176742090,0,1,3922.0,472.0,3154,1.698331e+09,...,16,2,0.602990,0,1,0,0,1,NaN,6.0


### Quelques stats de vérification des données

In [19]:
df_reviews_en["language"].value_counts()
print_voted_up_count_proportion(df_reviews_en)

bins = [0, 10, 20, 50, 100, 300, 512, float("inf")]
labels = ["<10", "10-20", "20-50", "50-100", "100-300", "300-512", ">512"]
df_reviews_en["tranche"] = pd.cut(df_reviews_en["word_count"], bins=bins, labels=labels)

print("\nAffichage de toute les tranches de longueurs:")
print(df_reviews_en["tranche"].value_counts().sort_index())

voted_up (reviews positive:1 / negative:0)
voted_up = 1: Count = 944482, Proportion = 76.18%
voted_up = 0: Count = 295351, Proportion = 23.82%

Pour obtenir une distribution équilibrée 50/50, en sous échantillonnant la classe majoritaire

On aurait 295351 reviews par classe. Soit un total de 590702 reviews.

Affichage de toute les tranches de longueurs:
tranche
<10        237244
10-20      148840
20-50      203851
50-100     174798
100-300    276001
300-512     99241
>512        99858
Name: count, dtype: int64


### Traitement du fichier en

In [20]:
df_reviews_clean_en = clean_review_with_all_processing(df_reviews_en)

print(f"Avant : {len(df_reviews_en)}")
print(f"Après : {len(df_reviews_clean_en)}")

print_voted_up_count_proportion(df_reviews_clean_en)

Avant : 1239833
Après : 500406
voted_up (reviews positive:1 / negative:0)
voted_up = 1: Count = 385855, Proportion = 77.11%
voted_up = 0: Count = 114551, Proportion = 22.89%

Pour obtenir une distribution équilibrée 50/50, en sous échantillonnant la classe majoritaire

On aurait 114551 reviews par classe. Soit un total de 229102 reviews.


### Vérification de quelques review positive

In [21]:
for i, review in enumerate(
    df_reviews_clean_en[
        (df_reviews_clean_en["voted_up"] == 1)
        & (df_reviews_clean_en["word_count"] <= 30)  # pas trop long pour relecture
    ]["review"].head(5),
    1,
):
    print(f"--- Review {i} ---")
    print(review)
    print()

--- Review 1 ---
if ur tired of cs2 come back to 2000 and play cs1.6

--- Review 2 ---
CS2 bring me here!!!!! i'd be glad if ur guys can bring back csgo

--- Review 3 ---
I started playing CS in 2002 with Version 1.5 and have since accumulated so many great memories over the years of playing the GOAT FPS.

--- Review 4 ---
best played with a membrane keyboard, a rollerball mouse, and a 800x600 CRT

--- Review 5 ---
It's a classic, still one the best games in the world!



### Vérification de quelques review négative

In [22]:
for i, review in enumerate(
    df_reviews_clean_en[
        (df_reviews_clean_en["voted_up"] == 0)
        & (df_reviews_clean_en["word_count"] <= 30)  # pas trop long pour relecture
    ]["review"].head(5),
    1,
):
    print(f"--- Review {i} ---")
    print(review)
    print()

--- Review 1 ---
Roadmap Simulator 2019 Snail paced development with constant delays and promises. To be perfectly frank, what they've released so far isn't that great.

--- Review 2 ---
This game is the reason I'll never buy an early access title again.

--- Review 3 ---
WRATH: Aeon of Delays Don't worry fanboys, I'll edit the review once the game actually releases.

--- Review 4 ---
I feel like 3D Realms scammed me out of 120$ that I paid more than 3 years ago for the founders edition of the game that will never come out.

--- Review 5 ---
Lol. Lmao. Just take down your road map at this point there's no reason to keep it up.



### Equilibrage des classes (50/50 positif/négatif)

In [23]:
df_balanced_en = balance_classes(df_reviews_clean_en)

print(f"\nDataset équilibré : {len(df_balanced_en)} reviews (50/50)")
print(df_balanced_en["voted_up"].value_counts())


Dataset équilibré : 229102 reviews (50/50)
voted_up
0    114551
1    114551
Name: count, dtype: int64


### Export du fichier des review en pour fine tuning

In [24]:
# export du fichier des reviews fr
output_file_en = os.path.join(path, "reviews_en_processed.csv")
keep_columns = ["voted_up", "review", "weighted_vote_score"]

df_balanced_en[keep_columns].to_csv(output_file_en, index=False, encoding="utf-8")
print(f"Fichier des reviews en nettoyées exporté vers : {output_file_en}")

Fichier des reviews en nettoyées exporté vers : /home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/reviews_en_processed.csv


## Vérification des fichiers crées

Les deux fichiers avec les review en et fr se trouvent dans le dossier cache kaggle du dataset initial (avec versionning)

In [25]:
list_files_recursively(path)

/home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/reviews_fr_processed.csv
/home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/weighted_score_above_08.csv
/home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/reviews_en_processed.csv
/home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/en_weighted_score_above_06.csv
/home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/fr_all_reviews.csv
/home/sebas/.cache/kagglehub/datasets/kieranpoc/steam-reviews/versions/2/all_reviews/all_reviews.csv
